# Laboratorio de regresión - 4

|                |   |
:----------------|---|
| **Nombre**     | Camial Daniela Zapata Castañeda  |
| **Fecha**      | 11 de Febrero del 2025  |
| **Expediente** | 745624  |

## Modelos penalizados

Hasta ahora la función de costo que usamos para decidir qué tan bueno es nuestro modelo al momento de ajustar es:

$$ \text{RSS} = \sum_{i=1}^n e_i^2 = \sum_{i=1}^n (y_i - \hat{y_i})^2 $$

Dado que los errores obtenidos son una combinación de sesgo y varianza, puede ser que se sesgue un parámetro para minimizar el error. Esto significa que el modelo puede decidir que la salida no sea una combinación de los factores, sino una fuerte predilección sobre uno de los factores solamente. 

E.g. se quiere ajustar un modelo

$$ \hat{z} = \hat{\beta_0} + \hat{\beta_1} x + \hat{\beta_2} y $$

Se ajusta el modelo y se decide que la mejor decisión es $\hat{\beta_1} = 10000$ y $\hat{\beta_2}=50$. Considera limitaciones de problemas reales:
- Quizás los parámetros son ajustes de maquinaria que se deben realizar para conseguir el mejor producto posible, y que $10000$ sea imposible de asignar.
- Quizás los datos actuales están sesgados y sólo hacen parecer que uno de los factores importa más que el otro.

Una de las formas en las que se puede mitigar este problema es penalizando a los parámetros del modelo, cambiando la función de costo:

$$ \text{RSS}_{L2} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p \hat{\beta_j}^2 $$

El *L2* significa que se está agregando una penalización de segundo orden. Lo que hace esta penalización es que los factores ahora sólo tendrán permitido crecer si hay una reducción al menos proporcional en el error (sacrificamos sesgo, pero reducimos la varianza).

Asimismo, existe la penalización *L1*

$$ \text{RSS}_{L1} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p |\hat{\beta_j}| $$

A las penalizaciones *L2* y *L1* se les conoce también como Ridge y Lasso, respectivamente.

Para realizar una regresión con penalización de Ridge o de Lasso usamos el objeto `Ridge(alpha=?)` o `Lasso(alpha=?)` en lugar de `LinearRegression()` de `sklearn`.

Utiliza el dataset de publicidad (Advertising.csv) y realiza 3 regresiones múltiples:

$$ \text{sales} = \beta_0 + \beta_1 (\text{TV}) + \beta_2 (\text{radio}) + \beta_3 (\text{newspaper}) + \epsilon $$

1. Sin penalización
2. Con penalización L2
3. Con penalización L1

Compara los resultados de los parámetros y sus *p-values*, y los $R^2$ resultantes.

In [98]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from scipy import stats

In [9]:
data = pd.read_csv("Advertising.csv")

In [10]:
data.head(3)

,Unnamed: 0,TV,radio,newspaper,sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3


In [11]:
data = data.drop(columns = ["Unnamed: 0"])

In [18]:
x = data.iloc[:, :-1].values

In [19]:
y = np.reshape(data["sales"].values, [-1, 1])

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.4, random_state = 2)

In [69]:
n = len(X_test)

In [22]:
scaler = StandardScaler()

In [23]:
scaler.fit(X_train)

StandardScaler()

In [108]:
X_std_test = scaler.transform(X_test)

### Sin penalización 

In [38]:
model = LinearRegression()

In [39]:
model = model.fit(X_std_train, y_train)

In [40]:
y_pred_test = model.predict(X_std_test)

#### Coeficientes

In [101]:
intercepto = model.intercept_
betas = model.coef_
intercepto, betas

(array([13.78875]), array([[ 3.89613949,  2.70744689, -0.00495557]]))

#### r_2

In [102]:
r2_test = r2_score(y_test, y_pred_test)
r2_test

0.8668365598374413

####  P_values

In [62]:
p = 4

In [109]:
ones = np.ones([n, 1])
X_std_test_1 = np.hstack((ones, X_std_test))

In [111]:
#Error Estadar de los coeficientes para la regresión sin penalización

RSS = np.sum((y_pred_test - y_test)**2)
sig = RSS/(n-p)


X_inv = np.linalg.inv(X_std_test_1.T @ X_std_test_1)
std_error_Sp = np.sqrt(np.diagonal(sig * X_inv))

print(std_error_Sp)

[0.17702478 0.16439446 0.19198209 0.17450881]


In [94]:
# valores t-estadístico 

t0 = intercepto/np.sqrt(std_error_Sp[0])
t1 = betas[0, 0]/np.sqrt(std_error_Sp[1])
t2 = betas[0, 1]/np.sqrt(std_error_Sp[2])
t3 = betas[0, 2]/np.sqrt(std_error_Sp[3])

print(t0, t1, t2, t3)

[32.772371] 9.609281418517819 6.179162032000547 -0.011862744608324375


In [100]:
# Sacamos los p_values

pvalue_b0 = 2*(1 - stats.t.cdf(np.abs(t0), n-p))
pvalue_b1 = 2*(1 - stats.t.cdf(np.abs(t1), n-p))
pvalue_b2 = 2*(1 - stats.t.cdf(np.abs(t2), n-p))
pvalue_b3 = 2*(1 - stats.t.cdf(np.abs(t3), n-p))

print(pvalue_b0, pvalue_b1, pvalue_b2, pvalue_b3)

[0.] 2.220446049250313e-16 1.0056493637833341e-08 0.9905556763611092


### Con penalización Ridge

In [104]:
Ridge = Ridge(alpha = 0.5)

In [105]:
Ridge = Ridge.fit(X_std_train, y_train)

In [112]:
y_pred_test_Ridge = Ridge.predict(X_std_test)

#### Coeficientes

In [135]:
intercepto_Ridge = Ridge.intercept_
betas_Ridge = Ridge.coef_

intercepto_Ridge, betas_Ridge

(array([13.78875]), array([[3.8720266 , 2.69148142, 0.00614817]]))

#### r_2

In [132]:
r2_test_Ridge = r2_score(y_test, y_pred_test_Ridge)
r2_test_Ridge

0.8671426653788139

#### P_values

In [122]:
#Error Estadar de los coeficientes para la regresión sin penalización

RSS_Ridge = np.sum((y_pred_test_Ridge - y_test)**2)
sig_Ridge = RSS_Ridge/(n-p)


X_inv = np.linalg.inv(X_std_test_1.T @ X_std_test_1)
std_error_Sp_Ridge = np.sqrt(np.diagonal(sig_Ridge * X_inv))

print(std_error_Sp_Ridge)

# valores t-estadístico 

t0_Ridge = interpecto_Ridge/np.sqrt(std_error_Sp_Ridge[0])
t1_Ridge = betas_Ridge[0, 0]/np.sqrt(std_error_Sp_Ridge[1])
t2_Ridge = betas_Ridge[0, 1]/np.sqrt(std_error_Sp_Ridge[2])
t3_Ridge = betas_Ridge[0, 2]/np.sqrt(std_error_Sp_Ridge[3])

print(t0_Ridge, t1_Ridge, t2_Ridge, t3_Ridge)

# Sacamos los p_values

pvalue_b0_Ridge = 2*(1 - stats.t.cdf(np.abs(t0_Ridge), n-p))
pvalue_b1_Ridge = 2*(1 - stats.t.cdf(np.abs(t1_Ridge), n-p))
pvalue_b2_Ridge = 2*(1 - stats.t.cdf(np.abs(t2_Ridge), n-p))
pvalue_b3_Ridge = 2*(1 - stats.t.cdf(np.abs(t3_Ridge), n-p))

print(pvalue_b0_Ridge, pvalue_b1_Ridge, pvalue_b2_Ridge, pvalue_b3_Ridge)

[0.1768212  0.1642054  0.19176131 0.17430812]
[32.79123174] 9.555306332736055 6.146259479775914 0.014726083214920806
[0.] 2.220446049250313e-16 1.1756593476519583e-08 0.9882762277714625


### Con penalización Lasso

In [124]:
Lasso = Lasso(alpha = 0.5)

In [125]:
Lasso = Lasso.fit(X_std_train, y_train)

In [126]:
y_pred_test_Lasso = Lasso.predict(X_std_test)

#### Coeficientes

In [127]:
intercepto_Lasso = Lasso.intercept_
betas_Lasso = Lasso.coef_

intercepto_Lasso, betas_Lasso

(array([13.78875]), array([3.47183185, 2.28275329, 0.        ]))

#### r_2

In [128]:
r2_test_Lasso = r2_score(y_test, y_pred_test_Lasso)
r2_test_Lasso

0.8567155028190123

#### P_values

In [131]:
#Error Estadar de los coeficientes para la regresión sin penalización

RSS_Lasso = np.sum((y_pred_test_Lasso - y_test)**2)
sig_Lasso = RSS_Lasso/(n-p)


X_inv = np.linalg.inv(X_std_test_1.T @ X_std_test_1)
std_error_Sp_Lasso = np.sqrt(np.diagonal(sig_Lasso * X_inv))

print(std_error_Sp_Lasso)

# valores t-estadístico 

t0_Lasso = intercepto_Lasso/np.sqrt(std_error_Sp_Lasso[0])
t1_Lasso = betas_Lasso[0]/np.sqrt(std_error_Sp_Lasso[1])
t2_Lasso = betas_Lasso[1]/np.sqrt(std_error_Sp_Lasso[2])
t3_Lasso = betas_Lasso[2]/np.sqrt(std_error_Sp_Lasso[3])

print(t0_Lasso, t1_Lasso, t2_Lasso, t3_Lasso)

# Sacamos los p_values

pvalue_b0_Lasso = 2*(1 - stats.t.cdf(np.abs(t0_Lasso), n-p))
pvalue_b1_Lasso = 2*(1 - stats.t.cdf(np.abs(t1_Lasso), n-p))
pvalue_b2_Lasso = 2*(1 - stats.t.cdf(np.abs(t2_Lasso), n-p))
pvalue_b3_Lasso = 2*(1 - stats.t.cdf(np.abs(t3_Lasso), n-p))

print(pvalue_b0_Lasso, pvalue_b1_Lasso, pvalue_b2_Lasso, pvalue_b3_Lasso)

[6.91954982 6.42585528 7.50420167 6.8212054 ]
[5.24186676] 1.3695983156465663 0.8333102599062119 0.0
[7.3020393e-07] 0.17348080827747037 0.4063971745549173 1.0


********************************************

#### Comparación de datos 

In [138]:
data = {
    "Regresión": [r2_test, intercepto, pvalue_b0, betas[0, 0], pvalue_b1, betas[0, 1], pvalue_b2, betas[0, 2], pvalue_b3],
    "Ridge": [r2_test_Ridge, intercepto_Ridge, pvalue_b0_Ridge, betas_Ridge[0, 0], pvalue_b1_Ridge, betas_Ridge[0, 1], pvalue_b2_Ridge, betas_Ridge[0, 2], pvalue_b3_Ridge],
    "Lasso": [r2_test_Lasso, intercepto_Lasso, pvalue_b0_Lasso, betas_Lasso[0], pvalue_b1_Lasso, betas_Lasso[1], pvalue_b2_Lasso, betas_Lasso[2], pvalue_b3_Lasso]
}

# Convertirlo en DataFrame y trasponerlo para cambiar filas y columnas
df = pd.DataFrame(data, index=["r_2", "Intercepto", "p_value", "TV", "p_valueTV", "radio", "p_valueRa", "NewsPaper", "p_valueNews"]).T

df

,r_2,Intercepto,p_value,TV,p_valueTV,radio,p_valueRa,NewsPaper,p_valueNews
Regresión,0.867143,[13.788749999999999],[0.0],3.896139,0.0,2.707447,0.0,-0.004956,0.990556
Ridge,0.867143,[13.788749999999999],[0.0],3.872027,0.0,2.691481,0.0,0.006148,0.988276
Lasso,0.856716,[13.788749999999999],[7.302039304324381e-07],3.471832,0.173481,2.282753,0.406397,0.0,1.0


Vemos como el r_2 es mayor en la regresión normal, lo cual ya se esperaba dado que estas sacrificando un mejor r_2 al tratar de nivelar los valores de beta con la penalización. los interceptos son los mismos. De ahí en más todos los valores son relativamente similares y solo es importante señalar que el modelo de Lasso vuelve cero el beta de Newspaper ya que no aporta información relevante al modelo.  